In [42]:
import sqlite3

# Path to your database file
db_path = "optuna.db"

# Connect to the database
conn = sqlite3.connect(db_path)

# Create a cursor object to interact with the database
cursor = conn.cursor()

# List all tables in the database
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
print("Tables:", tables)

# Close the connection
conn.close()



Tables: [('studies',), ('version_info',), ('study_directions',), ('study_user_attributes',), ('study_system_attributes',), ('trials',), ('trial_user_attributes',), ('trial_system_attributes',), ('trial_params',), ('trial_values',), ('trial_intermediate_values',), ('trial_heartbeats',), ('alembic_version',)]


In [43]:
import pandas as pd

# Connect to the database
conn = sqlite3.connect(db_path)

# Load a specific table into a DataFrame
df = pd.read_sql_query("SELECT * FROM trial_params;", conn)

# Display the DataFrame
print(df)

    # Save the updated DataFrame to the specified CSV file
df.to_csv('optunadb', index=False)

# Close the connection
conn.close()



     param_id  trial_id                param_name  param_value  \
0           1         2                        lr     0.002020   
1           2         2              weight_decay     0.000025   
2           3         2                   dropout     0.394411   
3           4         3                        lr     0.000386   
4           5         2               hidden_dims   128.000000   
..        ...       ...                       ...          ...   
139       140        16                 max_edges    70.000000   
140       141        15  channel_expansion_factor     2.000000   
141       142        16  channel_expansion_factor     4.000000   
142       143        15             num_neighbors    30.000000   
143       144        16             num_neighbors    30.000000   

                                     distribution_json  
0    {"name": "FloatDistribution", "attributes": {"...  
1    {"name": "FloatDistribution", "attributes": {"...  
2    {"name": "FloatDistribution", "

In [44]:
import torch

# Load the .pt file
data = torch.load("models/best_model_trial_8.pt")

# Check the type of the loaded object
print(type(data))

# If it's a dictionary, list the keys
if isinstance(data, dict):
    print("Keys in the .pt file:", data.keys())

<class 'collections.OrderedDict'>
Keys in the .pt file: odict_keys(['base_model.feat_encoder.time_encoder.w.weight', 'base_model.feat_encoder.time_encoder.w.bias', 'base_model.feat_encoder.feat_encoder.weight', 'base_model.feat_encoder.feat_encoder.bias', 'base_model.layernorm.weight', 'base_model.layernorm.bias', 'base_model.mlp_head.weight', 'base_model.mlp_head.bias', 'base_model.mixer_blocks.0.transformer_encoder.W_Q.weight', 'base_model.mixer_blocks.0.transformer_encoder.W_Q.bias', 'base_model.mixer_blocks.0.transformer_encoder.W_K.weight', 'base_model.mixer_blocks.0.transformer_encoder.W_K.bias', 'base_model.mixer_blocks.0.transformer_encoder.W_V.weight', 'base_model.mixer_blocks.0.transformer_encoder.W_V.bias', 'base_model.mixer_blocks.0.transformer_encoder.sdp_attn.scale', 'base_model.mixer_blocks.0.transformer_encoder.to_out.0.weight', 'base_model.mixer_blocks.0.transformer_encoder.to_out.0.bias', 'base_model.mixer_blocks.0.channel_layernorm.weight', 'base_model.mixer_blocks.0